<a href="https://colab.research.google.com/github/mafaldasalomao/multilabel_classification_pocus/blob/main/multilabel_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import suppporting libraries
import tarfile
import urllib.request as urllib2
import os
import cv2
from os import listdir
from os.path import isfile, join
import re
#Import deep learning libraries
import cv2
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import tqdm as tqdm
import tensorflow_hub as hub

#Import data analytics libraries
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import scipy.io
import pandas as pd
import seaborn as sns
#Import image visualization libraries
from PIL import *
from PIL import ImageFile
from PIL import Image
#System settings
ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ['WANDB_CONSOLE'] = 'off'
#Coloring for print outputs
class color:
   RED = '\033[91m'
   BOLD = '\033[1m'
   END = '\033[0m'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp  /content/drive/MyDrive/datasets/convex.zip /content/

In [4]:
!cp /content/drive/MyDrive/datasets/dataset_metadata.csv /content/

In [5]:
!unzip -q convex.zip -d /content/temp

In [6]:
path='/content/temp/convex'
annotation_path = "dataset_metadata.csv"
videos_list = os.listdir(path)

In [7]:
columns = ['Filename', 'Effusion', 'Consolidations', 'B-lines', 'A-lines', 'Pleural_line_irregularities', 'Air_bronchogram', 'Label']
base = pd.read_csv(annotation_path, usecols=columns)

In [8]:
base = base[base['Effusion'] != 'n/A']
base = base[base['Consolidations'] != 'n/A']
base = base[base['B-lines'] != 'n/A']
base = base[base['A-lines']  != 'n/A']
base = base[base['Pleural_line_irregularities']  != 'n/A']
base = base[base['Air_bronchogram']  != 'n/A']
base = base[base['Label']  != 'n/A']
#base = base[base['Label']  != 'regular']
#base = base[base['Label']  != 'Regular']
base['Label']= base['Label'].replace(['regular'], 'Regular')

base['Effusion'] = pd.to_numeric(base['Effusion'])
base.head()

,Filename,Label,Effusion,Consolidations,B-lines,A-lines,Pleural_line_irregularities,Air_bronchogram
0,Cov-Butterfly-COVID Lung 2,COVID-19,1.0,0.0,1.0,0.0,1.0,0.0
1,Cov-Butterfly-Skip Lesion,COVID-19,0.0,1.0,1.0,1.0,1.0,0.0
2,Cov-Butterfly-Confluent B lines,COVID-19,0.0,0.0,1.0,0.0,0.0,0.0
3,Reg-Butterfly-Normal Lung A lines,Regular,0.0,0.0,0.0,1.0,0.0,0.0
4,Reg-Butterfly-Normal Lung_Example 2,Regular,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
base  = base.dropna(axis = 0)
base.shape

(346, 8)

In [10]:
metadata = base;
metadata['Filename']

0               Cov-Butterfly-COVID Lung 2
1                Cov-Butterfly-Skip Lesion
2          Cov-Butterfly-Confluent B lines
3        Reg-Butterfly-Normal Lung A lines
4      Reg-Butterfly-Normal Lung_Example 2
                      ...                 
344                Cov-B_ConvexProb_score1
345               Cov-C_Convex_Prob_score2
346               Cov-D_Convex_Prob_score3
347                Cov-C_ConvexProb_score3
348                    Cov-MSU-SkipLesions
Name: Filename, Length: 346, dtype: object

In [11]:
#"Cov-Butterfly-COVID Lung 2" in metadata['Filename']
teste = metadata.loc[metadata.Filename == "Reg-Atlas-lungcurtain"]
len(teste)

1

In [24]:
effusion_list = []
consolid_list = []
aline_list = []
bline_list = []
pli_list = []
airb_list = []
filenames = []
y_labels = []
num_imgs_per_video = 10
path_imgs_train = '/content/drive/MyDrive/datasets/imgs'
#os.mkdir(path_imgs_train)
os.mkdir("/content/dataset")

In [13]:
IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model
num_imgs_per_video=10

In [ ]:

feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

In [ ]:
feature_extractor_layer.trainable = True

In [14]:
def plot_curves(history, title):
    fig_1 = plt.figure(figsize=(5, 3))
    epochs = range(1, len(history.history['loss'])+1)
    plt.plot(epochs, history.history['loss'], label="Train loss")
    plt.plot(epochs, history.history['val_loss'], label="Test Loss")
    plt.title("Training Loss Curve: "+title)
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend();
    fig_1.savefig(path_imgs_train+'/loss_'+title+'.png')
    plt.close(fig_1)
    
    fig_2 = plt.figure(figsize=(5, 3))
    epochs = range(1, len(history.history['accuracy'])+1)
    plt.plot(epochs, history.history['accuracy'], label="Train Acc")
    plt.plot(epochs, history.history['val_accuracy'], label="Test Acc")
    plt.title("Training Accuracy Curve: "+title)
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.legend();
    fig_2.savefig(path_imgs_train+'/acc_'+title+'.png')
    plt.close(fig_2)

## Data Augmentation


In [ ]:
!pip install -U git+https://github.com/albumentations-team/albumentations

In [ ]:
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout

In [ ]:
def criar_diretorio(caminho):
  if not os.path.exists(caminho):
    os.makedirs(caminho)

In [ ]:
from scipy.stats.distributions import alpha
def create_dataset_aug(img_altura=224, img_largura=224, augmentation=True):
    X_train, y_train = [], []
    for filename in videos_list:
        count=0
        #print(filename)
        vid_cap = cv2.VideoCapture(path+'/'+filename)
        total_frame_count= int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = total_frame_count/num_imgs_per_video
        success, image = vid_cap.read()
        metadata_cap = metadata.loc[metadata.Filename == filename.split('.')[0]]
        if len(metadata_cap) >0:
            while success:
                #print("Reading new frame...", success)
                if ((count+1) % int(step) == 0):
                    #print("Writing new frame...")
                    out = '/content/dataset/{}_{}.png'.format(filename.split('.')[0], count)
                    cv2.imwrite(out, image)
                    img = tf.keras.utils.load_img(out, target_size = (img_largura, img_altura, 3))
                    img = tf.keras.utils.img_to_array(img)
                    img= img/255.
                    #img = img.transpose(2,0,1)
                    X_train.append(img)
                    img_labels = [metadata['A-lines'][metadata_cap.index[0]],
                                    metadata['B-lines'][metadata_cap.index[0]],
                                    metadata['Effusion'][metadata_cap.index[0]],
                                    metadata['Consolidations'][metadata_cap.index[0]],
                                    metadata['Pleural_line_irregularities'][metadata_cap.index[0]]]
                                     #metadata['Air_bronchogram'][metadata_cap.index[0]]])
                    y_train.append(img_labels)
                    if augmentation:
                      aug = HorizontalFlip(p = 1.0)
                      augmentation = aug(image = img)
                      x1 = augmentation['image']
                      X_train.append(x1)
                      y_train.append(img_labels)

                      aug = OpticalDistortion(p = 1.0, distort_limit=2, shift_limit=0.5)
                      augmentation = aug(image = img)
                      x2 = augmentation['image']
                      X_train.append(x2)
                      y_train.append(img_labels)

                      # aug = ElasticTransform(p = 1.0, alpha=120, sigma=120*0.05, alpha_affine=120*0.03)
                      # augmentation = aug(image = img)
                      # x3 = augmentation['image']
                      # X_train.append(x3)
                      # y_train.append(img_labels)

                      # aug = GridDistortion(p = 1.0)
                      # augmentation = aug(image = img)
                      # x4 = augmentation['image']
                      # X_train.append(x4)
                      # y_train.append(img_labels)

                success, image = vid_cap.read()
                count += 1
    X = np.array(X_train)
    y = np.array(y_train)
    return X, y

In [15]:
def create_mydataset_all():
    X_train, y_train = [], []
    for filename in videos_list:
        count=0
        #print(filename)
        vid_cap = cv2.VideoCapture(path+'/'+filename)
        total_frame_count= int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = total_frame_count/num_imgs_per_video
        success, image = vid_cap.read()
        metadata_cap = metadata.loc[metadata.Filename == filename.split('.')[0]]
        if len(metadata_cap) >0:
            while success:
                #print("Reading new frame...", success)
                if ((count+1) % int(step) == 0):
                    #print("Writing new frame...")
                    out = '/content/dataset/{}_{}.png'.format(filename.split('.')[0], count)
                    cv2.imwrite(out, image)
                    img = tf.keras.utils.load_img(out, target_size = (224, 224, 3))
                    img = tf.keras.utils.img_to_array(img)
                    img= img/255.
                    #img = img.transpose(2,0,1)
                    X_train.append(img)
                    y_train.append([metadata['A-lines'][metadata_cap.index[0]],
                                    metadata['B-lines'][metadata_cap.index[0]],
                                    metadata['Effusion'][metadata_cap.index[0]],
                                    metadata['Consolidations'][metadata_cap.index[0]],
                                    metadata['Pleural_line_irregularities'][metadata_cap.index[0]]])
                                     #metadata['Air_bronchogram'][metadata_cap.index[0]]])
                success, image = vid_cap.read()
                count += 1
    X = np.array(X_train)
    y = np.array(y_train)
    return X, y

In [16]:
#X, y = create_dataset_aug()
X, y = create_mydataset_all()
#((1225, 224, 224, 3), (1225, 5))
num_folds=5
# Define the K-fold Cross Validator
#kfold = KFold(n_splits=num_folds, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.2, shuffle=True)
del X
del y
#X_train.shape, y_train.shape

In [ ]:
from keras import initializers
#tf.random.set_seed(42)
N_LABELS = 5
EPOCHS = 20
batch_size = 32
LR = 0.0001
acc_per_fold = []
loss_per_fold = []

In [ ]:
reconstructed_model = keras.models.load_model("my_h5_model.h5")

## K-fold train

In [ ]:
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(X, y):
  N_LABELS = 5
  EPOCHS = 30
  LR = 1e-4
  batch_size = 32
  ##################
  #2048-5 = 
  initializer = tf.keras.initializers.Identity()
  model_all_0 = tf.keras.Sequential([
      feature_extractor_layer_resnet,
      Dense(512, activation='relu', name='hidden_layer_1'),
      Dense(512, activation='relu', name='hidden_layer_2'),
      Dense(128, activation='relu', name='hidden_layer_3'),
      Dense(N_LABELS, activation='sigmoid', name='output')
  ])
  # Compile the model
  model_all_0.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])

  history = model_all_0.fit(X[train], y[train],
              batch_size=batch_size,
              epochs=EPOCHS,
              validation_data=(X[test], y[test]))

  # Generate generalization metrics
  scores = model_all_0.evaluate(X[test], y[test], verbose=1)
  print(f'Score for fold {fold_no}: {model_all_0.metrics_names[0]} of {scores[0]}; {model_all_0.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


## Backbone resnet101

In [17]:
feature_extractor_url_resnet = "https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5"
feature_extractor_layer_resnet = hub.KerasLayer(feature_extractor_url_resnet,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

In [ ]:
feature_extractor_url_eff = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"
feature_extractor_layer_eff = hub.KerasLayer(feature_extractor_url_resnet,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

In [ ]:
print("Thickness of the model:", len(feature_extractor_layer_resnet.weights))
#list(feature_extractor_layer_resnet.layers)

In [18]:
feature_extractor_layer_resnet.trainable = False

In [ ]:
#X, y = create_dataset_aug()
X, y = create_mydataset_all()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.2, shuffle=True)
X_train.shape, y_train.shape
del X
del y

In [ ]:
#tf.random.set_seed(42)

## Unit Test

In [20]:
dense = [2048, 1024, 512, 256, 128, 64, 32, 16, 10, 5]

In [ ]:
tested = 45
for i in range(len(dense)):
  for j in range(len(dense)-i):
    for k in range(len(dense)-j-i):
      for l in range(len(dense)-j-i-k):
        if tested >-1:
          print('Trained')
        else:
          title = '2048_'+str(dense[i])+'_'+str(dense[j+i])+'_'+str(dense[k+j+i])+'_'+str(dense[k+j+i+l])
          print(title)
        tested = tested-1

In [ ]:
tested = 46
for i in range(len(dense)):
  for j in range(len(dense)-i):
    for k in range(len(dense)-j-i):
      for l in range(len(dense)-j-i-k):
        if tested >-1:
          print('Trained')
        else:
          N_LABELS = 5
          EPOCHS = 30
          LR = 1e-4
          batch_size = 32
          ##################
          #2048-5 = 
          initializer = tf.keras.initializers.Identity()
          model_all_0 = tf.keras.Sequential([
              feature_extractor_layer_resnet,
              Dense(2048, activation='relu', name='hidden_layer_1'),
              Dense(dense[i], activation='relu', name='hidden_layer_2'),
              Dense(dense[j+i], activation='relu', name='hidden_layer_3'),
              Dense(dense[k+j+i], activation='relu', name='hidden_layer_4'),
              Dense(dense[k+j+i+l], activation='relu', name='hidden_layer_5'),
              Dense(N_LABELS, activation='sigmoid', name='output')
          ])
          # Compile the model

          model_all_0.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
            loss='binary_crossentropy',
            metrics=['accuracy'])

          history = model_all_0.fit(X_train, y_train,
                      batch_size=batch_size,
                      epochs=EPOCHS,
                      validation_data=(X_test, y_test))
          title = title = '2048_'+str(dense[i])+'_'+str(dense[j+i])+'_'+str(dense[k+j+i])+'_'+str(dense[k+j+i+l])+'_5'
          print("Model: "+title)
          print("=================================================================================")
          plot_curves(history, title)
        tested = tested -1

Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Trained
Epoch 1/30
39/39 [==============================] - 12s 248ms/step - loss: 0.6030 - accuracy: 0.1992 - val_loss: 0.5566 - val_accuracy: 0.1987
Epoch 2/30
39/39 [==============================] - 8s 219ms/step - loss: 0.3728 - accuracy: 0.3314 - val_loss: 0.2109 - val_accuracy: 0.4853
Epoch 3/30
39/39 [==============================] - 8s 213ms/step - loss: 0.1325 - accuracy: 0.4988 - val_loss: 0.1413 - val_accuracy: 0.5049
Epoch 4/30
39/39 [==============================] - 8s 207ms/step - loss: 0.0647 - accuracy: 0.5559 - val_loss: 0.0669 - val_accuracy: 0.4853
Epoch 5/30
39/39 [==============================] - 8s 

In [ ]:
!zip -r data.zip imgs/

In [ ]:

base_model.trainable = True
# Freeze all layers except for the
for layer in base_model.layers[:-10]:
  layer.trainable = False

## Experiments




### Approved model


In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.27525442838668823 - Accuracy: 52.768731117248535%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.20195697247982025 - Accuracy: 96.09120488166809%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.18232537806034088 - Accuracy: 56.02605938911438%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.23135672509670258 - Accuracy: 80.06535768508911%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.15090304613113403 - Accuracy: 57.189542055130005%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.13643944263458252 - Accuracy: 70.58823704719543%
----------------------------------------------------------------